In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
from transformers import CLIPTokenizer
from diffusers import StableDiffusionPipeline, DDPMScheduler
from torchvision import transforms
from torch.cuda.amp import GradScaler, autocast

In [10]:
class LogoDataset(Dataset):
    def __init__(self, csv_path, image_folder, tokenizer, image_size=(256, 256)):
        self.data = pd.read_csv(csv_path)
        self.image_folder = image_folder
        self.tokenizer = tokenizer
        self.transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        prompt = row['Description']
        image_path = f"{self.image_folder}/{row['Name']}"

        tokens = self.tokenizer(prompt, padding="max_length", truncation=True, max_length=77, return_tensors="pt")
        image = Image.open(image_path).convert("RGB")
        image = self.transform(image)

        return tokens['input_ids'].squeeze(), tokens['attention_mask'].squeeze(), image

In [11]:
csv_path = "data.csv"
image_folder = "dataset"
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")

In [12]:
dataset = LogoDataset(csv_path, image_folder, tokenizer)
limited_dataset = torch.utils.data.Subset(dataset, range(10))
dataloader = DataLoader(limited_dataset, batch_size=1, shuffle=True)

In [13]:
model = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")
model.scheduler = DDPMScheduler.from_config(model.scheduler.config)

Loading pipeline components...: 100%|██████████| 7/7 [00:26<00:00,  3.86s/it]


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.vae.to(device)
model.unet.to(device)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 2.00 GiB of which 0 bytes is free. Of the allocated memory 8.36 GiB is allocated by PyTorch, and 175.26 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
model.unet.enable_gradient_checkpointing()

optimizer = torch.optim.AdamW(model.unet.parameters(), lr=5e-6)
scaler = GradScaler()

C:\Users\rajendrak\AppData\Local\Temp\ipykernel_23164\1704126386.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [ ]:
for epoch in range(5):
    for step, (input_ids, attention_mask, images) in enumerate(dataloader):
        input_ids, attention_mask, images = (
            input_ids.to(device),
            attention_mask.to(device),
            images.to(device),
        )
        optimizer.zero_grad()

        with torch.no_grad():
            latents = model.vae.encode(images).latent_dist.sample() * 0.18215
            noise = torch.randn_like(latents)
            timesteps = torch.randint(0, model.scheduler.num_train_timesteps, (latents.size(0),), device=device).long()
            noisy_latents = model.scheduler.add_noise(latents, noise, timesteps)

        with autocast():
            encoder_hidden_states = model.text_encoder(input_ids.to("cpu"))[0].to(device)
            model_output = model.unet(noisy_latents, timesteps, encoder_hidden_states).sample
            loss = torch.nn.functional.mse_loss(model_output, noise)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        del latents, noise, noisy_latents  # Free memory
        torch.cuda.empty_cache()

        if step % 10 == 0:
            print(f"Epoch {epoch}, Step {step}, Loss: {loss.item()}")

c:\Users\rajendrak\AppData\Local\Programs\Python\Python311\Lib\site-packages\PIL\Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
c:\Users\rajendrak\AppData\Local\Programs\Python\Python311\Lib\site-packages\diffusers\configuration_utils.py:140: FutureWarning: Accessing config attribute `num_train_timesteps` directly via 'DDPMScheduler' object attribute is deprecated. Please access 'num_train_timesteps' over 'DDPMScheduler's config object instead, e.g. 'scheduler.config.num_train_timesteps'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)
C:\Users\rajendrak\AppData\Local\Temp\ipykernel_23164\1456694249.py:16: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


OutOfMemoryError: CUDA out of memory. Tried to allocate 58.00 MiB. GPU 0 has a total capacity of 2.00 GiB of which 0 bytes is free. Of the allocated memory 8.31 GiB is allocated by PyTorch, and 211.92 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)